In [17]:
import os, shutil, re
import torch
import numpy as np
import argparse
from tqdm import tqdm


SEED = 1234
np.random.seed(SEED)
rng = np.random.RandomState(SEED)

# ARGS
HALF_WINDOW = 3

args = {}
args['dir'] = r'C:\Users\likit\Downloads\shhs'

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

DATASET_SUBJECTS = sorted(os.listdir(os.path.join(args['dir'], 'subjects_data')))
DATASET_SUBJECTS = [os.path.join(args['dir'], 'subjects_data', f) for f in DATASET_SUBJECTS]
TRAIN_PATH = os.path.join(args['dir'], f'train_{HALF_WINDOW}') 
TEST_PATH = os.path.join(args['dir'], f'test_{HALF_WINDOW}')

dataset_subjects_data = [np.load(f) for f in DATASET_SUBJECTS]
sub_train = rng.choice(DATASET_SUBJECTS, int(len(DATASET_SUBJECTS)*0.8), replace=False)
sub_test = list(set(DATASET_SUBJECTS) - set(sub_train))





In [2]:
import wandb, os
import numpy as np
import torch
from torch.utils.data import DataLoader

from utils.utils import *
from utils.dataloader import distillDataset
from helper_train import distill_train

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

DATASET = 'shhs'
MODALITY = 'eeg'
EXPERIMENT_NAME = f'supervised_{DATASET}+{MODALITY}'
INPUT_CHANNELS = 2
BATCH_SIZE = 256
EPOCH_LEN = 7

DATASET_PATH = r'C:\Users\likit\Downloads\shhs'
DATASET_SUBJECTS = os.listdir(os.path.join(DATASET_PATH, 'subjects_data'))
SAVE_PATH = './saved_weights'
if not os.path.exists(SAVE_PATH): os.makedirs(SAVE_PATH, exist_ok=True)

# wandb = wandb.init(
#     project="distillECG",
#     name=EXPERIMENT_NAME,
#     save_code=False,
#     entity="sleep-staging",
# )
# wandb.save("./supervised/utils/*")
# wandb.save("./supervised/models/*")
# wandb.save("./supervised/helper_train.py")
# wandb.save("./supervised/train.py")

DATASET_SUBJECTS.sort(key=natural_keys)
DATASET_SUBJECTS = [os.path.join(DATASET_PATH, 'subjects_data', f) for f in DATASET_SUBJECTS]
dataset_subjects_data = [np.load(f) for f in DATASET_SUBJECTS]

# load files
TRAIN_PATH = os.path.join(DATASET_PATH, f'train_{EPOCH_LEN}') 
TEST_PATH = os.path.join(DATASET_PATH, f'test_{EPOCH_LEN}')
TRAIN_EPOCH_FILES = [os.path.join(TRAIN_PATH, f) for f in os.listdir(TRAIN_PATH)]
TEST_EPOCH_FILES = [os.path.join(TEST_PATH, f) for f in os.listdir(TEST_PATH)]


print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print(f"Train files: {len(TRAIN_EPOCH_FILES)} \n")
print(f"Test files: {len(TEST_EPOCH_FILES)} \n")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

# train_dataset = distillDataset(TRAIN_EPOCH_FILES, MODALITY)
# test_dataset = distillDataset(TEST_EPOCH_FILES, MODALITY)

# train_loader = DataLoader(
#     train_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     num_workers=4,
# )
# test_loader = DataLoader(
#     test_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=False,
#     num_workers=4,
# )

# model = distill_train(EXPERIMENT_NAME, SAVE_PATH, train_loader, test_loader, wandb, EPOCH_LEN, INPUT_CHANNELS)
# wandb.watch([model], log="all", log_freq=500)

# model.fit()
# wandb.finish()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Train files: 8263 

Test files: 3141 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [13]:
import numpy as np
from torch.utils.data import Dataset

class d(Dataset):
    """Dataset for train and test"""

    def __init__(self, filepath, modality):
        super(d, self).__init__()
        self.file_path = filepath
        self.modality = modality

    def __len__(self):
        return len(self.file_path)

    def __getitem__(self, index):
        path = self.file_path[index]
        data = np.load(path)
        return data, data['y']

test_dataset = d(TEST_EPOCH_FILES, MODALITY)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)


In [15]:
import multiprocessing

multiprocessing.cpu_count()

12

In [12]:
for i in TEST_EPOCH_FILES:
    a = np.load(i)
    print(type(a['eeg']))
    break

<class 'numpy.ndarray'>


In [25]:
import os
import numpy as np
import argparse
from tqdm import tqdm

SEED = 1234
rng = np.random.RandomState(SEED)


# ARGS
HALF_WINDOW = 3 # Epoch length is HALF_WINDOW*2 + 1
AVAILABLE_MODALITY = ['eeg', 'ecg', 'eog', 'emg', 'emog']

args = {}
args['dir'] = r"C:\Users\likit\Downloads\shhs"


In [26]:


DATASET_SUBJECTS = sorted(os.listdir(os.path.join(args['dir'], 'subjects_data')))
DATASET_SUBJECTS = [os.path.join(args['dir'], 'subjects_data', f) for f in DATASET_SUBJECTS]
TRAIN_PATH = os.path.join(args['dir'], f'train_{HALF_WINDOW}') 
TEST_PATH = os.path.join(args['dir'], f'test_{HALF_WINDOW}')

dataset_subjects_data = [np.load(f) for f in DATASET_SUBJECTS]
sub_train = rng.choice(DATASET_SUBJECTS, int(len(DATASET_SUBJECTS)*0.8), replace=False)
sub_test = list(set(DATASET_SUBJECTS) - set(sub_train))
train_data = [np.load(f) for f in sub_train]
test_data = [np.load(f) for f in sub_test]

print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print(f"Train subjects: {len(sub_train)} \n")
print(f"Test subjects: {len(sub_test)} \n")
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

if not os.path.exists(TRAIN_PATH): os.makedirs(TRAIN_PATH, exist_ok=True)
if not os.path.exists(TEST_PATH): os.makedirs(TEST_PATH, exist_ok=True)



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Train subjects: 8 

Test subjects: 3 

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [36]:
path = r'C:\Users\likit\Downloads\shhs\edfs\shhs1\shhs1-200975.edf'

import mne

a = mne.io.read_raw_edf(path, include='ECG')

a.ch_names


Extracting EDF parameters from C:\Users\likit\Downloads\shhs\edfs\shhs1\shhs1-200975.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


['ECG']